### 15.066 Optimization Final Project Proposal (July 2021)
Lauren Heintz, Gabriel Pascualy, Logan Greene, Lisa Schleuter, Jomi Kramer, Nick Esposito  
LGO 2023  
August 14, 2021

<h1><center>OFF GRID HOUSING ENERGY</center></h1> 

## 1. Introduction

As the effects of global warming become increasingly apparent, people are considering living off the grid. Living off the grid means that you do not rely on public utilities for utilities like electricity, running water, sewer, natural gas, or heat, but rather you create your own energy. Currently, most people take a hybrid approach meaning that they utilize small capacity renewable energy products while still utilizing some public utilities.  

For many Americans, the deciding factor to transition to low-carbon electricity is that the cost of energy from these products must be cheaper than the energy produced by the grid coupled with the environmental impact associated with the implementation.  In the past, the cost of renewable energy products was very expensive making it financially impractical to make the transition due to high initial costs for the equipment. Over time, the financial benefit of renewable energy products can save a substantial amount of money on utilities while vastly decreasing the impact on the environment. Just a decade ago, it was much cheaper to build a new coal-burning power plant than it was to build a large capacity wind or solar plant. Since then, the price per MWh has declined by 89% for large capacity renewable energy sources which has trickled down into the smaller capacity renewable energy product market (Roser, 2020)1. Recently, reductions in the initial cost of renewable energy products have been spurred on by improvements in manufacturing processes and advances in technology which have made living off the grid more accessible to a wider range of income levels. As of 2017, approximately 180,000 families live off the grid which is expected to significantly increase every year (Infoplease, 2020)2. 

Our project aims to assist Americans considering living off the grid. We plan to develop a tool to help minimize the overall cost associated with off-grid products for a family’s energy needs based on the energy resources available. Depending on their location, people can input variables into the tool, such as average monthly electricity and heat demand, initial and monthly budget, and costs associated with renewable energy products and grid information. Based on these parameters, the tool will output the best combination of products (including utilizing the grid) and the optimal number of units of each product to meet demand while minimizing financial and environmental costs.  

## 2. Background
####  Demand
This off-grid housing energy optimization problem starts first with energy demand data. We chose to use electric, hourly demand data over the course of the year in units of kwh (kilowatt-hours). This data was procured from << website/source? >> which gives total grid energy consumption for various regions of the United States. Using information on the percent breakdown of residential energy usage from the grid, and census data on the population of the region, we derived an average hourly energy consumption metric per household, per hour, for every hour of a standard 365 day calendar year, using the readily available data for year 2020 (removing February 29th since 2020 is a leap year). This estimation method aligned closely with the total estimated energy consumption per household from << EIA - US Energy Information Administration >>, so we decided this was an adequate metric to use. 

<< we can insert an equation here to represent the math if we want >>

Then, the average hourly electricity demand for each day was utilized in the model to capture the changes in demand during different seasons and for different parts of the country with different weather patterns. The user interface in this notebook allows the user to choose from a few predefined locations in a drop-down list which capture demand data from different regions of the United States. Users can also procure and upload their own demand data in the format required for the model. 

Additionally, we provide a feature which allows you to choose the times of year to consider demand data for your home. For a full time resident, this would be the full year. However, if this off-grid house is a summer vacation home or a winter vacation home, that may change your energy demand needs. As a result, the recommended product suggestions output from our model will change according to this data.

<< insert some sample graphs/charts of demand data from somewhere showing a seasonal pattern >>

A complementary set of data required to supplement this energy demand data is data describing the efficiency of different energy generation products based on location in the world. The efficiency of solar and wind can vary greatly based on the time of year, the amount of sunlight a region recieves, temperature, and local weather patterns for that area. We acquired data on the efficiency of solar power and wind power from <<< website here >> which allows an input of any city, and in return provides hourly efficiency data. This data is used in the optimization to understand the optimal mix of energy products to support your home, depending on the location of your home. 

<< insert visuals on efficiency data, if we want to >>

#### Supply 
In our optimization model, the supply of energy to meet the demand comes from several possible power sources. These can first be divided by on-grid and off-grid power sources. Within off-grid power sources, we have energy generation products and energy storage products. The two types of energy generation products wind and solar power. The table below defines the mix of products we chose to input in to our model, so the output of the model provides an optimal combination of these power sources that meets energy demand and other constraints, while minimizing cost to the resident.   
  

<center> Available Products </center>

Product |Generation/Storage |Energy Type |On-Grid/Off-Grid
-----|-----|-----|----| 
**Grid Power**|Generation|Grid|On-Grid
**Solar XS**|Generation|Solar|Off-Grid
**Solar S**|Generation|Solar|Off-Grid
**Solar M**|Generation|Solar|Off-Grid
**Solar L**|Generation|Solar|Off-Grid  
**Wind Turbine S**|Generation|Wind|Off-Grid
**Wind Turbine L**|Generation|Wind|Off-Grid
**Battery**|Storage|Storage|Off-Grid

<center> Table 1 </center>

Each of these products is defined with a max amount of energy that it can provide in the units of kwh (kilowatt-hours). This is defined in a configuration CSV file which can be uploaded by the user for custom products they would like to consider. Each power generation product (solar and wind) has a maximum capacity in kwh, which is multipled by an efficiency metric mentioned in the demand section above. This provides the expected hourly power generation in kwh of each product over every hour of the year.

The energy generation products and storage products interact directly with each other. "Storage" products here are batteries, and then can be used to store excess energy generated, or they can be used to meet the energy demand of the customer. Our system is set up so that excess energy generated on a daily basis from wind and solar products can be stored indefinitely and used for other days when demand is higher. We operate under the assumptions that you would not pull energy from the grid and store it on your batteries, and that the intention of using battery energy is to minimze the amount of on-grid energy consumed.

Because a household's ability to satisfy demand requirements is a combination of their energy generation products and their storage products, the output of our tool is a suggestion on number of products recommended for the user to buy from the list given in Table 1 above.   
   
<< graph of line plots showing: demand, energy generated, energy stored, and grid power over the same x-axis of time >>


<< pictures of turbine, solar panel, battery >>>>

#### Cost

<u>Off Grid Product Cost Structure</u>  

Each product from the product list has a set of cost variables associated with it that are defined below:
- **"Opening Cost"**: The baseline set up cost to install the wiring and necessary equipment to connect an external energy source (solar, wind, batteries) to a house in a way that can be used by the household. This would only be done once for each type of energy source.
- **"Unit Cost"**: The cost per unit of a product, such as a solar panel or a wind turbine, that is incurred once per unit at the time of initial purchase.
- **"Maintenance Cost"**: The estimated annual cost to upkeep and maintain a given product from the product list.

<u>Grid Connection Cost Structure</u> 
- **"Opening Cost"**: The baseline set up cost to install the wiring and necessary equipment to connect the house the nearest grid power source, assuming that it is not yet connected.
- **"Grid Hourly Cost"**: The cost per kilowatt-hour to pull energy directly from the grid and use it in a home.

**Delete below? Original Notes**  
Demand:  
- Different locations (in a drop down list)
- show plots of solar efficiency and wind efficiency
- plots on weather
- different times of year (seasonal patterns)

Supply:
- explain solar (specs, etc)
- explain wind (specs, etc)
- explain batteries (specs, etc) 
- discussion the sample output from this tool: we want to know what mix of products and storage to buy
- discussion on the energy generation products and the storage needed, how it interacts
- explain all the storage variables here
- *discussion/assumption of battery starting partially full *** 
- *graph of line plots showing: demand, energy generated, energy stored, and grid power over the same x-axis of time*
- *pictures of turbine, solar panel, battery*

Cost:
- define a generic power source and the cost structure associated with it
- ic, mc, oc

## 3. Theoretical Formulation

### A. Demand
Let $h$ := be the hour of the year where total hours in a year = (24*365), $h \in H$ where $H = \{{0, 1, 2, ... 8760}\}$

Let $d_{h}$:= be the customer electricity demand in KwH during hour $h$ of the year where $d_{h} \geq 0$

### B. Power Source Parameters
##### Product Definition Parameters
Let $j$ := be a product $j$ $\in$ $J$ where $J = \{{"smallsolar", "largesolar", "smallwind", "largewind", "smallbattery","grid"}\}$

Let $E$ := be a set containing indices of products which generate electricity except for "grid"

Let $S$ := be a set containing indices of products which store electricity 

Let $et$ := be an energy type where $et \in T = \{{"solar", "wind", "storage"}\}$

Let $ET_{et}$ := be a set containing all products of energy type $et$

##### Cost Paramaters
Let $oc_{j}$ := be the opening cost in US dollars of installing required infrastructure for product $j$ which belongs to an energy type $et$

Let $uc_j$ := be the unit cost in US dollars of purchasing an additional unit of a specific product $j$ where $ic_j \geq 0$

Let $mc_j$ := be the cost in US dollars of maintaining one unit of product $j$ where $mc_j \geq 0$

Let $Gc_h$ := be the cost of on-grid power in units of dollers per kilowatt hour for hour $h$  

Let $Gr$ := be the revenue rate in dollars per KwH that excess energy can be sold back to the grid. This number represents positive revenue.

##### Energy Conversion & Storage Parameters
Let $\rho_{h,j}$ := be the efficiency of product $j$ during hour $h$ of the year where $\rho_{h,j}$ is continuous from 0 to 1

Let $c_{j}$ := be the capacity of product $j$ in KwH if it generates or stores electricity


### C. Decision Variables
##### Product Decision Variables
Let $x_j$ := be a binary variable representing whether or not the opening cost of $j$'s energy type is paid for where $x_j \in$ {0,1}

Let $y_{j}$ := be an integer variable representing the number of units of product $j$ where $y_j \geq$ 0

##### Grid Power Decision Variables
Let $g_{h}$ := be the continuous variable that represents the amount of power needed from the grid in kilowatt-hours during hour $h$ of the year.

Let $ge_{h}$ := be the continuous variable that represents the amount of excess energy sold back to the grid in kilowatt-hours during hour $h$ of the year.

##### Storage Decision Variables
Let $es_{h,j}$ := be the continuous variable that represents the amount of energy stored by product $j$ (note: not all products can store energy)

Let $sc_{h,j}$ := be the continuous variable that represents the amount of stored consumed from product $j$ (note: not all products can store energy)

### D. Calculated Variables
##### Calculated Storage Variables
Let $ES_{h}$ := be the total energy stored this year through hour $h$ calculated by summing cumulative the energy stored each hour for all products $\sum_{j \in J} es_{h,j}$

Let $SC_{h}$ := be the total storage consumed this year through hour $h$ calculated by summing the cumulative energy consumed each hour for all products $\sum_{j \in J} sc_{h,j}$

Let $TT$ := be the total energy storage of the system calculated by summing the capacity of all products $\sum_{j \in J} c_{j}y_{j}$



### E. Objective Function
We want to minimize the cost; therefore, the objective function is the sum of various costs:

    a) opening cost (oc) the cost of initially setting up a new service

    b) unit cost (uc)
    
    c) maintenance cost (mc)

    d) cost of grid/hr (Gc) * grid hours consumed (g)

after subtracting the revenue from excess energy sold back to the grid:
    
    e) grid sell back rate (Gr) * energy sold to grid (ge)

$$\min \sum_{j \in J} oc_{j}x_j + \sum_{j \in J} uc_jy_j + \sum_{j \in J} mc_{j}y_{j} + \sum_{h \in H} {(Gc_h)(g_{h}) - (Gr)(ge_h)} $$

### F. Constraints

##### Demand Constraints
1) Meet customer demand for electricity per hour per month by summing the energy generation products ($E$) output, the stored energy ($S$) available, and grid power ($g$) available

$$ \forall h \in H:  \sum_{j\in E} \rho_{h,j} y_{j} + \sum_{j\in S}sc_{h,j} + g_{h} \geq d_{h}$$

##### Budget Constraints
2) You have an initial budget of the amount of capital you have on hand to shell out for initial set up.
$$  \sum_{j \in J} oc_{j}x_j + \sum_{j \in J} ic_jy_j \leq $200,000 $$

3) You also have a monthly budget.
$$ \sum_{j \in J} mc_{j}y_{j} \leq $500 $$

##### Set up and Installation Constraints
4) Ensure at least 1 $x_j$ per energy type is 1 when any $y_j$ of the energy type is $\geq 1$

$$ \forall j \in ET_{solar}: \sum_{j' \in ET_{solar}} x_{j'} * M >= y_{j}$$

$$ \forall j \in ET_{wind}: \sum_{j' \in ET_{wind}} x_{j'} * M >= y_{j}$$

$$ \forall j \in ET_{storage}: \sum_{j' \in ET_{storage}} x_{j'} * M >= y_{j}$$

5) Ensure that at most 1 $x_j$ per energy type is equal to 1

$$ \forall et \in ET: \sum_{j\in ET_{et}}x_j \leq 1$$

##### Storage Constraints
6) Total storage consumed at hour $h$ of the year cannot exceed electricity stored

$$ \forall h \in H\space: SC_{h} \leq ES_{h} $$ 

7) Storage available at hour $h$ of the year cannot exceed total storage capacity

$$ \forall h \in H\space: ES_{h} - SC_{h}  \leq TT $$

8) Electricity stored at hour $h$ of the year cannot exceed the sum of electricity produced, storage consumed, and grid usage minus demand at hour $h$ of the year and energy sold back to the grid at hour $h$. 

$$ \forall h \in H:\space \sum_{j\in S} es_{h,j} \leq g_{h} - d_{h} - ge_h + \sum_{j\in E} \rho_{h,j} y_{j} + \sum_{j\in S} sc_{h,j} $$

##### Grid Constraints

9) No energy can be sold back to the grid unless the grid is connected/installed to the house.

$$ \forall h \in H:\space ge_h \leq x_{j=grid}*M $$

###### do we need one more BigM constraint here that forces the grid Xi to be turned on, or is that covered??

## 4. Interface/Gui
- description of features & capability
- hosted on binder and accessible via github
- explanation of the visuals included in the output

## 5. Results
#### Sub Experiments
- compare the results of off grid cost over 1, 3, 5, 10, 20, 30 years to the cost of just grid power for those times (graphs)
- take total grid cost over 50 years (something larger than the amortization period), add constraint so that total cost of system over 50 years is equal to the total grid cost. 
    - make amortization a decision variable so that you can solve the minimization to minimize your amortization period to figure out the break even point
    - Solving for the break even point in terms of # of years, and factoring amortization in to that
- show different results for a few different regions of the country (differing demand)
- relax or tighten budget constraints (cheap, medium, expensive option)
    - show variation in budget and resulting plots of energy generated, energy needed from grid, mix of products purchased (as output by our optimization)
- run some variations where it is a seasonal vacation home and only use the demand for that period and see the results
- shadow prices
    - try to look at the shadow prices of the capital budget
    - shadow price of specific products?
- by adjusting the capital budget, see how the break even points varies (in terms of years)
- in discussion of results, use the size of the solar panels and space needed for wind turbines to output how much sqft or acreage is needed to support this amount of product
    - could also be implemented as a constraint too but TBD for now

## 6. Optimization Tool

In [2]:
from offgridoptimizer.interface import OffGridOptimizer
ogo = OffGridOptimizer()
# ogo.load_sheets(None)
# ogo.optimize(None)
ogo.interface

ModuleNotFoundError: No module named 'tabulate'